In [1]:
from bids import BIDSLayout
import os

In [2]:
data_csv = '/lustre06/project/6061841/mwang8/data/.tabular/ppmi/bagel.csv'
data_path = '/home/niusham/projects/rrg-jbpoline/mwang8/data/.bids/ppmi/'

In [5]:
layout = BIDSLayout(data_path)

In [ ]:
# layout.get()

In [34]:
path_to_HC = "../HC_selected_subjects.txt"
path_to_PD = "../PD_selected_subjects.txt"

In [38]:
with open(path_to_HC, "r") as f:
    list_HC = []
    for l in f:
        list_HC.append(l.strip().replace("sub-",""))


In [39]:
with open(path_to_PD, "r") as f:
    list_PD = []
    for l in f:
        list_PD.append(l.strip().replace("sub-",""))

In [41]:
path_list_HC = layout.get(subject=list_HC ,extension='nii.gz', run=1, session="BL" ,suffix='T1w', return_type='filename')

In [44]:
path_list_PD = layout.get(subject=list_PD ,extension='nii.gz', run=1, session="BL" ,suffix='T1w', return_type='filename')

In [46]:
with open("../HC_paths.txt", "w") as f:
    for p in path_list_HC:
        f.write(f"{p}\n")

In [45]:
with open("../PD_paths.txt", "w") as f:
    for p in path_list_PD:
        f.write(f"{p}\n")